In [ ]:
import pandas as pd
import pymysql
import requests
import datetime
import time

In [ ]:
def market_index(date):
    url = f'https://www.twse.com.tw/fund/exchangeReport/FMTQIK?response=json&date={date}'
    response = requests.get(url)
    data = response.json()
    
    if data['stat'] == 'OK':
        df = pd.DataFrame.from_dict(data["data"])  
        df.columns = data["fields"]
        return df

In [ ]:
# Open database connection
db = pymysql.connect(
host='127.0.0.1', port=3306, user='root', passwd='dish0263', db='stock', charset='utf8')

cursor = db.cursor()

create = """ CREATE TABLE market_index(
                日期 DATE, 
                成交股數 VARCHAR(20), 
                成交金額 VARCHAR(20), 
                成交筆數 VARCHAR(20), 
                發行量加權股價指數 VARCHAR(20), 
                漲跌點數 VARCHAR(20))
         """

insert = """INSERT INTO market_index(
                日期, 
                成交股數, 
                成交金額, 
                成交筆數, 
                發行量加權股價指數, 
                漲跌點數) VALUES (%s,%s,%s,%s,%s,%s)
         """

drop = """DROP TABLE market_index"""

In [ ]:
#創建 table
cursor.execute(create)

In [ ]:
#新增資料
for i in range(len(df)):
    x = tuple(df.loc[i])
    cursor.execute(insert,x)

In [ ]:
#儲存變更
db.commit()

In [ ]:
def stock(date):
    url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY_ALL?response=json&date={date}&type=MS'
    res = requests.get(url)
    data = res.json()
    
    if data['stat'] == 'OK':
        df = pd.DataFrame.from_dict(data["data"])  
        df.columns = data["fields"]
        df.insert(0,'日期',data['date'])
        return df

In [ ]:
create = """ CREATE TABLE stock(
                日期 DATE, 
                證券代號 VARCHAR(20),
                成交股數 VARCHAR(20),
                證券名稱 VARCHAR(20), 
                成交金額 VARCHAR(20), 
                開盤價 VARCHAR(20), 
                最高價 VARCHAR(20),
                最低價 VARCHAR(20),
                收盤價 VARCHAR(20),
                漲跌價差 VARCHAR(20),
                成交筆數 VARCHAR(20))
         """

insert = """INSERT INTO stock(
              日期, 
              證券代號,
              證券名稱,
              成交股數,
              成交金額,
              開盤價,
              最高價,
              最低價,
              收盤價,
              漲跌價差,
              成交筆數) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
         """
drop = """DROP TABLE stock"""

In [ ]:
#創建 table
cursor.execute(create)

In [ ]:
#獲取歷史資料

start = '2022-2-16'
today = datetime.date.today()

#yesterday = datetime.date.today() + datetime.timedelta(days=-1)

for i in pd.date_range(start, today)[::-1]:
    print(i.strftime('%Y%m%d'))
    df = stock(i.strftime('%Y%m%d'))
    time.sleep(10)
    for i in range(len(df)):
        x = tuple(df.loc[i])
        cursor.execute(insert,x)

In [ ]:
#儲存變更
db.commit()

In [ ]:
#股票清單
res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
df = pd.read_html(res.text)[0]

# 設定column名稱

df.columns = df.iloc[0]

# 刪除前兩行
df = df.iloc[2:]

# 欄位分割並重設
df.insert(0,'證券代號',df['有價證券代號及名稱'].str[:4])
df.insert(0,'證券名稱',df['有價證券代號及名稱'].str[5:].str.replace('T',''))

# 刪除無用欄位
df = df.drop(columns=['備註','有價證券代號及名稱'])